In [3]:
from pymongo import MongoClient
from utils import utils
config = utils.get_config()
server = "192.168.2.68"
port = '27018'

CONFIG = ["H16","config"]

client = MongoClient(server + ':' + port)
config = list(client[CONFIG[0]][CONFIG[1]].find())
client.close()
if len(config) == 0:
    config['app']['dr_events_auto_accept']
else:
    config[0]['auto_answer']

OSError: Caravels config file not setup!

In [1]:
from datetime import datetime, timedelta
from pymongo import MongoClient, DESCENDING

def get_iots(IOTS_READING):
    server = "192.168.2.68"
    port = '27018'

    start = datetime.now().replace(hour=0,minute=0, second=0, microsecond=0) - timedelta(days=7)
    end = start + timedelta(days=1)

    client = MongoClient(server + ':' + port)
    historic = list(client[IOTS_READING[0]][IOTS_READING[1]].find({'datetime': {'$gt': start, '$lt': end}}))
    client.close()
    return historic

In [3]:
import pandas as pd

def get_type_sum(TYPE, IOTS_READING):
    historic = get_iots(IOTS_READING)
    total = []
    for step in historic:
        iots = []
        for iot in step["iots"]:
            if iot['type'] == TYPE:
                for value in iot['values']:
                    if value['type'] == 'power':
                        iots.append(value['values'])
        total.append([sum(iots),step['datetime']])
    total = pd.DataFrame(total, columns=['consumption','datetime'])
    total['datetime'] = pd.to_datetime(total['datetime'], format='%Y-%m-%d %H:%M:%S', dayfirst=True)
    total.set_index("datetime", inplace=True)
    total = total.resample('1h').mean()
    return total

IOTS_READING_LEFT = ["BuildingLeftSide","iots_reading_left_side"]
IOTS_READING_RIGHT = ["BuildingRightSide","iots_reading"]
total_r = get_type_sum('hvac',IOTS_READING_RIGHT)
total_r

,consumption
datetime,
2024-05-16 00:00:00,7.233333
2024-05-16 01:00:00,8.290278
2024-05-16 02:00:00,5.641667
2024-05-16 03:00:00,6.533333
2024-05-16 04:00:00,6.233333
2024-05-16 05:00:00,5.066667
2024-05-16 06:00:00,5.873611
2024-05-16 07:00:00,6.041667
2024-05-16 08:00:00,5.897222


In [3]:
total_l = get_type_sum('hvac',IOTS_READING_LEFT)
total_l

,consumption
datetime,
2024-04-04 00:00:00,14.113287
2024-04-04 01:00:00,13.533333
2024-04-04 02:00:00,13.154167
2024-04-04 03:00:00,14.250000
2024-04-04 04:00:00,13.335188
2024-04-04 05:00:00,13.604167
2024-04-04 06:00:00,14.051389
2024-04-04 07:00:00,13.399166
2024-04-04 08:00:00,13.787500


In [4]:
total = total_r
total['consumption-l'] = total_l['consumption']
total

,consumption,consumption-l
datetime,,
2024-04-04 00:00:00,5.614525,14.113287
2024-04-04 01:00:00,6.373611,13.533333
2024-04-04 02:00:00,6.650000,13.154167
2024-04-04 03:00:00,4.590278,14.250000
2024-04-04 04:00:00,5.783333,13.335188
2024-04-04 05:00:00,4.533333,13.604167
2024-04-04 06:00:00,5.005556,14.051389
2024-04-04 07:00:00,5.638889,13.399166
2024-04-04 08:00:00,9.005556,13.787500


In [4]:
total = get_type_sum('sockets',IOTS_READING_RIGHT)
total

,consumption
datetime,
2024-05-16 00:00:00,94.530556
2024-05-16 01:00:00,94.895833
2024-05-16 02:00:00,94.062500
2024-05-16 03:00:00,93.993056
2024-05-16 04:00:00,93.922222
2024-05-16 05:00:00,93.563889
2024-05-16 06:00:00,95.072222
2024-05-16 07:00:00,93.876389
2024-05-16 08:00:00,94.504167


In [6]:
total = get_type_sum('sockets',IOTS_READING_LEFT)
total

,consumption
datetime,
2024-05-16 00:00:00,1533.819444
2024-05-16 01:00:00,1530.166898
2024-05-16 02:00:00,1533.222222
2024-05-16 03:00:00,1530.403338
2024-05-16 04:00:00,1537.263889
2024-05-16 05:00:00,1530.138889
2024-05-16 06:00:00,1525.285118
2024-05-16 07:00:00,1525.666667
2024-05-16 08:00:00,1538.247566


In [7]:
total = get_type_sum('lamp',IOTS_READING_RIGHT)
total

,consumption
datetime,
2024-05-16 00:00:00,0.0
2024-05-16 01:00:00,0.0
2024-05-16 02:00:00,0.0
2024-05-16 03:00:00,0.0
2024-05-16 04:00:00,0.0
2024-05-16 05:00:00,0.0
2024-05-16 06:00:00,0.0
2024-05-16 07:00:00,0.0
2024-05-16 08:00:00,0.0


In [8]:
total = get_type_sum('lamp',IOTS_READING_LEFT)
total

,consumption
datetime,
2024-05-16 00:00:00,0.000000
2024-05-16 01:00:00,0.000000
2024-05-16 02:00:00,0.000000
2024-05-16 03:00:00,0.000000
2024-05-16 04:00:00,0.000000
2024-05-16 05:00:00,0.000000
2024-05-16 06:00:00,0.000000
2024-05-16 07:00:00,0.000000
2024-05-16 08:00:00,0.000000


In [13]:
from datetime import datetime, timedelta
from pymongo import MongoClient, DESCENDING
import pandas as pd

BATTERIES = ["BuildingLeftSide","batteries"]
server = "192.168.2.68"
port = '27018'

def get_batteries_historic(start):
    client = MongoClient(server + ':' + port)
    historic = list(client[BATTERIES[0]][BATTERIES[1]].find(
        {'datetime': {'$gt': datetime.strptime(start, "%Y-%m-%d %H:%M:%S")}}))
    client.close()
    return historic
total = get_batteries_historic(datetime.strftime(datetime.now() - timedelta(hours=24), "%Y-%m-%d %H:%M:%S"))
total = pd.DataFrame(total)
total = total.drop(["_id"], axis=1)
total = total.dropna()
total

,batteries,datetime
0,"[{'name': 'Battery_4', 'values': [{'type': 'en...",2024-04-10 11:03:58.489
1,"[{'name': 'Battery_4', 'values': [{'type': 'en...",2024-04-10 11:04:03.483
2,"[{'name': 'Battery_4', 'values': [{'type': 'en...",2024-04-10 11:04:08.497
3,"[{'name': 'Battery_4', 'values': [{'type': 'en...",2024-04-10 11:04:13.500
4,"[{'name': 'Battery_4', 'values': [{'type': 'en...",2024-04-10 11:04:18.494
...,...,...
17262,"[{'name': 'Battery_4', 'values': [{'type': 'en...",2024-04-11 11:03:30.498
17263,"[{'name': 'Battery_4', 'values': [{'type': 'en...",2024-04-11 11:03:35.479
17264,"[{'name': 'Battery_4', 'values': [{'type': 'en...",2024-04-11 11:03:40.463
17265,"[{'name': 'Battery_4', 'values': [{'type': 'en...",2024-04-11 11:03:45.483


In [14]:

def get_batteries():
    batteries = [
    {
        "name": "Battery_4",
        "type": "battery",
        "ip": "192.168.2.57",
        "capacity": 3600,
        "values" : [
            {
                "type" : "energy",
                "tag" : "battery.stored_energy",
                "data" : "DOUBLE"
            },
            {
                "type" : "energy",
                "tag" : "battery.charging_rate",
                "data" : "DOUBLE"
            }
        ]
    },
    {
        "name": "Battery_5",
        "type": "battery",
        "ip": "192.168.2.58",
        "capacity": 2400,
        "values" : [
            {
                "type" : "energy",
                "tag" : "battery.stored_energy",
                "data" : "DOUBLE"
            },
            {
                "type" : "energy",
                "tag" : "battery.charging_rate",
                "data" : "DOUBLE"
            }
        ]
    },
    {
        "name": "Battery_6",
        "type": "battery",
        "ip": "192.168.2.59",
        "capacity": 3600,
        "values" : [
            {
                "type" : "energy",
                "tag" : "battery.stored_energy",
                "data" : "DOUBLE"
            },
            {
                "type" : "energy",
                "tag" : "battery.charging_rate",
                "data" : "DOUBLE"
            }
        ]
    }]
    return batteries

In [15]:
total = total.values.tolist()

batteries_energy = []
for row in total:
    batteries = row[0]
    date = row[1]
    stored_energy = []
    config = get_batteries()
    
    a = []
    for battery in batteries:
        for value in battery['values']:
            if 'values' in value:
                if value['tag'] == "battery.stored_energy":
                    stored_energy.append(value['values'])
    aux = [date]
    
    aux.extend(stored_energy)
    batteries_energy.append(aux)
    
batteries_energy 

[[Timestamp('2024-04-10 11:03:58.489000'), 100, 100, 100],
 [Timestamp('2024-04-10 11:04:03.483000'), 100, 100, 99],
 [Timestamp('2024-04-10 11:04:08.497000'), 99, 100, 99],
 [Timestamp('2024-04-10 11:04:13.500000'), 99, 100, 99],
 [Timestamp('2024-04-10 11:04:18.494000'), 99, 100, 99],
 [Timestamp('2024-04-10 11:04:23.462000'), 99, 99, 99],
 [Timestamp('2024-04-10 11:04:28.520000'), 100, 99, 99],
 [Timestamp('2024-04-10 11:04:33.492000'), 100, 99, 99],
 [Timestamp('2024-04-10 11:04:38.428000'), 100, 99, 100],
 [Timestamp('2024-04-10 11:04:43.353000'), 100, 99, 100],
 [Timestamp('2024-04-10 11:04:48.461000'), 100, 99, 100],
 [Timestamp('2024-04-10 11:04:53.490000'), 100, 99, 100],
 [Timestamp('2024-04-10 11:04:58.498000'), 100, 99, 100],
 [Timestamp('2024-04-10 11:05:03.506000'), 100, 100, 100],
 [Timestamp('2024-04-10 11:05:08.477000'), 100, 100, 100],
 [Timestamp('2024-04-10 11:05:13.494000'), 100, 100, 100],
 [Timestamp('2024-04-10 11:05:18.493000'), 100, 100, 100],
 [Timestamp('202

In [16]:

names = ['datetime', 'stored_energy']

batteries = total[0][0]
for battery in batteries:
    names.append(battery['name'])

total = pd.DataFrame(batteries_energy, columns=names)
total['datetime'] = pd.to_datetime(total['datetime'], format='%Y-%m-%d %H:%M:%S', dayfirst=True)
total.set_index("datetime", inplace=True)
total = total.resample('1h').mean()
total = total.tail(24)
total['datetime'] = total.index
total = total.values.tolist()

ValueError: 5 columns passed, passed data had 4 columns